In [ ]:
#이 소스코드는 time_slding_window_realtime 전에 사용됩니다. 
#시간 기반 슬라이딩 윈도우를 위해 서버에서 들어오는 데이터를 10초 간격으로 csv파일로 저장합니다.

import socket
import sys
import csv
import datetime
from datetime import timedelta
import time

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('192.168.0.108', 9999)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)

#슬라이딩 윈도우 시간 설정
starttime = datetime.datetime.now()
endtime = starttime + timedelta(seconds=10)
windowlist = []
    
while True:
    data = sock.recv(16) #16 문자열을 받아옴
    if data:
        timenow = datetime.datetime.now()
        msg = [timenow,data.decode('utf-8')]
        
        if timenow <= endtime and timenow >= starttime:
            windowlist.append(msg)
            #윈도우 리스트 내에 데이터를 10초 동안 저장함
            
        elif timenow <= endtime+timedelta(seconds=1) and timenow >= starttime+timedelta(seconds=1):
            #1초마다 윈도우 첫시간과 끝시간이 이동함
            starttime = starttime + timedelta(seconds=1)
            endtime = endtime + timedelta(seconds=1)
            windowlist.append(msg)
            windowlist = list(filter(lambda x:x[0]>starttime,windowlist)) #윈도우 첫시간보다 낮은 데이터 시간 삭제
            with open('C:/Users/Shin/Documents/testbeddata/result/10se_window.csv', 'w') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerows(windowlist)
                csvFile.close()
        else:
            #마지막 데이터가 들어온 시간이 윈도우 크기를 넘어설 경우
            endtime = timenow
            starttime = timenow - timedelta(seconds=10)
            windowlist.append(msg)
            windowlist = list(filter(lambda x:x[0]>starttime,windowlist)) #윈도우 첫시간보다 낮은 데이터 시간 삭제
            with open('C:/Users/Shin/Documents/testbeddata/result/10se_window.csv', 'w') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerows(windowlist)
                csvFile.close()        